In [2]:
!pip install --quiet google-api-python-client pandas openpyxl tqdm

In [6]:
# 설정

from googleapiclient.discovery import build     # YouTube API 호출을 위한 라이브러리
from googleapiclient.errors import HttpError    # API 호출 시 발생하는 오류 처리용
import pandas as pd                             # CSV 파일 읽기 및 데이터 처리
from tqdm import tqdm                           # 진행 상황(progress bar) 표시
import json, os, time                           # JSON 저장, 경로 관리, 대기 시간 처리

# YouTube Data API 키
API_KEY = "" #API KEY 입력

# 입력 CSV 파일 경로
INPUT_CSV   = "/content/videos_with_dates.csv"

# 추출한 댓글 데이터를 저장할 JSON 파일 경로
OUTPUT_JSON = "/content/youtube_comments.json"

# API 요청 실패나 오류 발생 시 기록할 JSON 파일 경로
FAIL_JSON   = "/content/youtube_comment_failures.json"

# 각 영상에서 가져올 댓글 수 (TOP_N개의 최상위 댓글만 가져옴)
TOP_N          = 10

# 댓글 정렬 기준 ('time' = 최신순, 'relevance' = 인기/관련도 순)
ORDER          = "relevance"

# 처리할 영상 개수 제한 (None이면 CSV의 모든 영상 처리, 숫자 지정 시 일부만 처리)
LIMIT_VIDEOS   = None

# API 요청 사이의 대기 시간(초).
# YouTube API 쿼터 초과 방지를 위해 0.05~0.2초 사이로 설정 가능.
SLEEP_BETWEEN  = 0.0

In [7]:
# 유틸 함수

def fetch_comments_top_n(youtube, video_id: str, top_n: int = 10, order: str = "relevance"):
    rows = []  # 댓글 정보를 저장할 리스트

    try:
        # YouTube API 요청
        res = youtube.commentThreads().list(
            part="snippet",          # 댓글 메타데이터 포함
            videoId=video_id,        # 대상 영상 ID
            maxResults=100,          # 요청 한 번에 몇개의 댓글을 받아올건지.
            order=order,             # 'time' or 'relevance'
            textFormat="plainText"   # HTML 태그 없이 순수 텍스트로
        ).execute()
    except HttpError as e:
        # API 호출 실패 시 예외를 그대로 외부로 전달
        raise e

    # 응답에서 댓글 리스트 추출
    items = res.get("items", [])
    for it in items:
        snip = it["snippet"]["topLevelComment"]["snippet"]
        rows.append({
            "video_id": video_id,
            "comment_id": it["snippet"]["topLevelComment"]["id"],
            "text": snip.get("textDisplay"),                # 댓글 내용
            "published_at": snip.get("publishedAt"),        # 댓글 작성 시간
            "like_count": int(snip.get("likeCount", 0) or 0)
        })

    # 좋아요 수와 작성 시간 기준으로 내림차순 정렬
    rows.sort(key=lambda r: (r["like_count"], r["published_at"] or ""), reverse=True)

    # 요청된 필드만 반환
    return [{
        "video_id": r["video_id"],
        "comment_id": r["comment_id"],
        "text": r["text"],
        "published_at": r["published_at"],
        #좋아요 수도 보고싶으면 아래 주석 빼기
        #"like_count": r["like_count"]
    } for r in rows[:top_n]]


In [8]:
# 3) 메인: CSV 읽고 → 각 video_id 처리 → JSON 저장
#    (실패는 별도 JSON 로그로 저장)

# 입력 CSV 읽기
df = pd.read_csv(INPUT_CSV)

# 컬럼명을 모두 소문자로 변환 (대소문자 차이 방지)
df.columns = [c.lower() for c in df.columns]

# video_id 컬럼이 존재하는지 확인
assert "video_id" in df.columns, "입력 CSV에 'video_id' 컬럼이 필요합니다."

# video_id 목록 추출 (NaN 제거 → 문자열 변환 → 중복 제거)
video_ids = (
    df["video_id"].dropna()
    .astype(str)
    .drop_duplicates()
    .tolist()
)

# LIMIT_VIDEOS 값이 있으면 상위 일부만 처리
if LIMIT_VIDEOS:
    video_ids = video_ids[:LIMIT_VIDEOS]

# YouTube API 클라이언트 객체 생성
youtube = build("youtube", "v3", developerKey=API_KEY)

# 결과 저장용 리스트
out = []   # 성공한 댓글 데이터
fail = []  # 실패한 영상 로그

# 각 영상별로 댓글 수집
for vid in tqdm(video_ids, desc="Fetching", unit="video"):
    try:
        # 댓글 수집 (상위 N개)
        rows = fetch_comments_top_n(
            youtube, vid, top_n=TOP_N, order=ORDER
        )
        out.extend(rows)  # 결과 누적

    except HttpError as e:
        # YouTube API 관련 오류 처리
        # 예: commentsDisabled(403), notFound(404) 등
        fail.append({
            "video_id": vid,
            "status": getattr(e.resp, "status", None),  # HTTP 상태코드
            "reason": "commentsDisabled" if (
                getattr(e.resp, "status", None) == 403
                and "commentsDisabled" in str(e)
            ) else "HttpError",  # 원인 구분
            "message": str(e)[:500]  # 에러 메시지(최대 500자)
        })

    except Exception as e:
        # 예외(파일 문제, 데이터 처리 오류 등) 처리
        fail.append({
            "video_id": vid,
            "status": None,
            "reason": "Exception",
            "message": str(e)[:500]
        })

    # 과도한 요청 방지를 위한 대기
    if SLEEP_BETWEEN > 0:
        time.sleep(SLEEP_BETWEEN)

# --------------------------------
# 결과 저장 (JSON Pretty 출력, UTF-8)
# --------------------------------
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

# 실패 로그 저장 (실패 건이 있을 때만)
if fail:
    with open(FAIL_JSON, "w", encoding="utf-8") as f:
        json.dump(fail, f, ensure_ascii=False, indent=2)

# 처리 요약 출력
print(f"Saved: {OUTPUT_JSON}  (rows={len(out)})")
print(f"Failures: {len(fail)}", ("-> " + FAIL_JSON) if fail else "")

Fetching: 100%|██████████| 3/3 [00:00<00:00,  3.27video/s]

Saved: /content/youtube_comments.json  (rows=30)
Failures: 0 
